# Lab 9

Question A1. Continue your project work from last week’s exercise. Experiment with various optimizers and report your results with loss values, accuracies, precisions, recalls and F-scores.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

2025-04-11 13:26:21.200596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744377981.477242      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744377981.551166      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force TensorFlow to use CPU only
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'   # Suppress TensorFlow warnings and errors

# ----------------------------------------
# ✅ Paths and Constants
# ----------------------------------------
train_dir = '/kaggle/input/ecg-analysis/ECG_DATA/train'
test_dir = '/kaggle/input/ecg-analysis/ECG_DATA/test'
IMG_SIZE = (180, 180)
BATCH_SIZE = 32
EPOCHS = 10
SEED = 42

In [3]:
# ----------------------------------------
# ✅ Load Data using ImageDataGenerator
# ----------------------------------------
def load_data():
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_data = datagen.flow_from_directory(
        train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', seed=SEED)

    test_data = datagen.flow_from_directory(
        test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
        class_mode='categorical', shuffle=False, seed=SEED)
    
    return train_data, test_data

In [4]:
# ----------------------------------------
# ✅ Define CNN Model Architecture
# ----------------------------------------
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),  # Add this to fix warning
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [5]:
# ----------------------------------------
# ✅ Train Model with Given Optimizer
# ----------------------------------------
def train_with_optimizer(optimizer_name, train_data, test_data):
    print(f"\n🔧 Training with optimizer: {optimizer_name.upper()}\n")

    input_shape = IMG_SIZE + (3,)
    num_classes = train_data.num_classes
    model = build_cnn_model(input_shape, num_classes)

    # Select optimizer
    optimizer_dict = {
        'adam': tf.keras.optimizers.Adam(),
        'sgd': tf.keras.optimizers.SGD(),
        'rmsprop': tf.keras.optimizers.RMSprop()
    }

    model.compile(optimizer=optimizer_dict[optimizer_name],
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train model
    history = model.fit(train_data, epochs=EPOCHS,
                        validation_data=test_data, verbose=1)

    # Predict and evaluate
    predictions = model.predict(test_data)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_data.classes
    class_labels = list(test_data.class_indices.keys())

    # Print classification report
    print(classification_report(y_true, y_pred, target_names=class_labels))

    return history

# ----------------------------------------
# ✅ Main Execution: Try All Optimizers
# ----------------------------------------
if __name__ == "__main__":
    train_data, test_data = load_data()
    for optimizer in ['adam', 'sgd', 'rmsprop']:
        train_with_optimizer(optimizer, train_data, test_data)

Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.

🔧 Training with optimizer: ADAM



2025-04-11 13:26:41.352395: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.3575 - loss: 2.2042 - val_accuracy: 0.8955 - val_loss: 0.4036
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.8412 - loss: 0.4602 - val_accuracy: 0.9763 - val_loss: 0.1384
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.9274 - loss: 0.2156 - val_accuracy: 0.9914 - val_loss: 0.0531
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.9446 - loss: 0.1553 - val_accuracy: 0.9989 - val_loss: 0.0139
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.9781 - loss: 0.0650 - val_accuracy: 1.0000 - val_loss: 0.0067
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.9782 - loss: 0.0564 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.9895 - loss: 0.0318 - val_accuracy: 1.0000 - val_loss: 0.0035
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.9894 - loss: 0.0423 - val_accuracy: 1.0000 - val_loss: 0.

# Lab 10

Question A1. Continue your project work from last week’s exercise. Experiment with various regularization & optimization techniques and report your results with loss values, accuracies, precisions, recalls and F-scores.  

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

In [7]:
# Environment setup
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Constants
train_dir = '/kaggle/input/ecg-analysis/ECG_DATA/train'
test_dir = '/kaggle/input/ecg-analysis/ECG_DATA/test'
IMG_SIZE = (180, 180)
BATCH_SIZE = 32
EPOCHS = 10

In [8]:
# Load data
def load_data():
    datagen = ImageDataGenerator(rescale=1./255)
    train = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
    test = datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)
    return train, test

# CNN with L2 and Dropout
def build_regularized_cnn(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [9]:
# Train & Evaluate
def train_regularized_model(optimizer_name):
    train_data, test_data = load_data()
    input_shape = IMG_SIZE + (3,)
    num_classes = train_data.num_classes
    model = build_regularized_cnn(input_shape, num_classes)

    optimizers = {
        'adam': tf.keras.optimizers.Adam(),
        'rmsprop': tf.keras.optimizers.RMSprop()
    }

    model.compile(optimizer=optimizers[optimizer_name],
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_data, epochs=EPOCHS, validation_data=test_data, verbose=1)
    
    predictions = model.predict(test_data)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_data.classes
    class_labels = list(test_data.class_indices.keys())

    print(classification_report(y_true, y_pred, target_names=class_labels))

# Run
if __name__ == '__main__':
    for opt in ['adam', 'rmsprop']:
        print(f"\n🔧 Training with {opt.upper()} (with L2 & Dropout)\n")
        train_regularized_model(opt)


🔧 Training with ADAM (with L2 & Dropout)

Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.2913 - loss: 2.7330 - val_accuracy: 0.3782 - val_loss: 1.4806
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.4564 - loss: 1.3592 - val_accuracy: 0.8244 - val_loss: 0.7951
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.7417 - loss: 0.7851 - val_accuracy: 0.9310 - val_loss: 0.4356
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.8324 - loss: 0.5690 - val_accuracy: 0.9741 - val_loss: 0.2437
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.8621 - loss: 0.4460 - val_accuracy: 0.9881 - val_loss: 0.2119
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.9018 - loss: 0.3836 - val_accuracy: 0.9935 - val_loss: 0.1763
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.9157 - loss: 0.3321 - val_accuracy: 0.9968 - val_loss: 0.1441
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.9192 - loss: 0.3084 - val_accuracy: 0.9849 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.3149 - loss: 4.1873 - val_accuracy: 0.5162 - val_loss: 1.3797
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.5657 - loss: 1.2095 - val_accuracy: 0.8567 - val_loss: 0.5850
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.8356 - loss: 0.5794 - val_accuracy: 0.9267 - val_loss: 0.3315
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.9107 - loss: 0.3412 - val_accuracy: 0.9871 - val_loss: 0.2289
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.9660 - loss: 0.2253 - val_accuracy: 0.9989 - val_loss: 0.1349
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9733 - loss: 0.1948 - val_accuracy: 0.9440 - val_loss: 0.2413
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.9788 - loss: 0.1722 - val_accuracy: 0.9644 - val_loss: 0.1799
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.9785 - loss: 0.1420 - val_accuracy: 1.0000 - val_loss: 0.

Question A2. Study about RNN and learn to implement a basic RNN. Identify how to use RNN for your project work. 

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Reshape, Input
from sklearn.metrics import classification_report

2025-04-11 16:47:51.054891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744390071.317355      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744390071.391210      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
# Environment config
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Config
train_dir = '/kaggle/input/ecg-analysis/ECG_DATA/train'
test_dir = '/kaggle/input/ecg-analysis/ECG_DATA/test'
IMG_SIZE = (180, 180)
BATCH_SIZE = 32
EPOCHS = 50

In [14]:
# Load data
def load_data():
    datagen = ImageDataGenerator(rescale=1./255)
    train = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
    test = datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)
    return train, test

# Basic RNN model (reshape image to sequences)
def build_rnn(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),          # ✅ input_shape = (180, 540)
        SimpleRNN(64, activation='tanh'),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [15]:
# Train and evaluate RNN
def train_rnn_model():
    train_data, test_data = load_data()
    num_classes = train_data.num_classes

    # Flatten images for RNN input
    X_train, y_train = [], []
    for i in range(len(train_data)):
        x, y = train_data[i]
        X_train.append(x.reshape(x.shape[0], IMG_SIZE[0], IMG_SIZE[1]*3))
        y_train.append(y)
        if len(X_train)*BATCH_SIZE >= train_data.samples:
            break
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)

    X_test, y_test = [], []
    for i in range(len(test_data)):
        x, y = test_data[i]
        X_test.append(x.reshape(x.shape[0], IMG_SIZE[0], IMG_SIZE[1]*3))
        y_test.append(y)
        if len(X_test)*BATCH_SIZE >= test_data.samples:
            break
    X_test = np.concatenate(X_test)
    y_test = np.concatenate(y_test)

    model = build_rnn((IMG_SIZE[0], IMG_SIZE[1]*3), num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_test, y_test), verbose=1)

    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)
    class_labels = list(test_data.class_indices.keys())
    print(classification_report(y_true, y_pred, target_names=class_labels))

# Run
if __name__ == '__main__':
    train_rnn_model()

Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.
Epoch 1/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.2910 - loss: 1.4282 - val_accuracy: 0.2575 - val_loss: 1.3788
Epoch 2/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.2989 - loss: 1.3667 - val_accuracy: 0.2575 - val_loss: 1.3930
Epoch 3/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3173 - loss: 1.3642 - val_accuracy: 0.3060 - val_loss: 1.3756
Epoch 4/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3004 - loss: 1.3711 - val_accuracy: 0.2575 - val_loss: 1.3937
Epoch 5/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.3116 - loss: 1.3601 - val_accuracy: 0.2575 - val_loss: 1.3833
Epoch 6/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.3298 - loss: 1.3590 - val_accuracy: 0.3060 - val_loss: 1.3779
Epoch 7/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.2908 - loss: 1.3721 - val_accuracy: 0.2575 - val_loss: 1.3977
Epoch 8/50
95/95 ━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Lab 11

Question A1. Continue your project work from last week’s exercise. Upon implementation of RNN for your project work, analyze the impact created on the results compared to the CNN results. 

In [16]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, SimpleRNN
from sklearn.metrics import classification_report

In [17]:
# ----------------------------------------
# ✅ Configurations
# ----------------------------------------
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

IMG_SIZE = (180, 180)
BATCH_SIZE = 32
EPOCHS = 5  # Reduced for quick comparison
train_dir = '/kaggle/input/ecg-analysis/ECG_DATA/train'
test_dir = '/kaggle/input/ecg-analysis/ECG_DATA/test'

In [18]:
# ----------------------------------------
# ✅ Data Loading (Shared for CNN & RNN)
# ----------------------------------------
def load_data(flatten_for_rnn=False):
    datagen = ImageDataGenerator(rescale=1./255)
    train_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
    test_gen = datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

    if not flatten_for_rnn:
        return train_gen, test_gen
    
    # For RNN - Flatten images into sequences (height, width*channels)
    def flatten_batches(generator):
        X, y = [], []
        for i in range(len(generator)):
            x_batch, y_batch = generator[i]
            x_flat = x_batch.reshape(x_batch.shape[0], IMG_SIZE[0], IMG_SIZE[1]*3)
            X.append(x_flat)
            y.append(y_batch)
            if len(X)*BATCH_SIZE >= generator.samples:
                break
        return np.concatenate(X), np.concatenate(y)

    X_train, y_train = flatten_batches(train_gen)
    X_test, y_test = flatten_batches(test_gen)
    return X_train, y_train, X_test, y_test, train_gen.class_indices

# ----------------------------------------
# ✅ CNN Model
# ----------------------------------------
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# ----------------------------------------
# ✅ RNN Model
# ----------------------------------------
def build_rnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        SimpleRNN(64, activation='tanh'),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [19]:
# ----------------------------------------
# ✅ Evaluation Helper
# ----------------------------------------
def evaluate_model(model, X, y, label_map):
    y_pred = np.argmax(model.predict(X), axis=1)
    y_true = np.argmax(y, axis=1)
    class_names = list(label_map.keys())
    print(classification_report(y_true, y_pred, target_names=class_names))

# ----------------------------------------
# ✅ Run CNN
# ----------------------------------------
def run_cnn():
    print("\n🚀 Running CNN...")
    train_data, test_data = load_data()
    input_shape = IMG_SIZE + (3,)
    num_classes = train_data.num_classes
    model = build_cnn_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_data, epochs=EPOCHS, validation_data=test_data, verbose=1)

    # Evaluate
    test_data.reset()  # Ensure same order
    y_true = test_data.classes
    y_pred = np.argmax(model.predict(test_data), axis=1)
    print("\n📊 CNN Classification Report:\n")
    print(classification_report(y_true, y_pred, target_names=list(test_data.class_indices.keys())))

# ----------------------------------------
# ✅ Run RNN
# ----------------------------------------
def run_rnn():
    print("\n🚀 Running RNN...")
    X_train, y_train, X_test, y_test, label_map = load_data(flatten_for_rnn=True)
    input_shape = (IMG_SIZE[0], IMG_SIZE[1]*3)
    num_classes = y_train.shape[1]

    model = build_rnn_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_test, y_test), verbose=1)

    # Evaluate
    print("\n📊 RNN Classification Report:\n")
    evaluate_model(model, X_test, y_test, label_map)

# ----------------------------------------
# ✅ Main
# ----------------------------------------
if __name__ == "__main__":
    run_cnn()
    run_rnn()


🚀 Running CNN...
Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.3028 - loss: 2.0410 - val_accuracy: 0.2575 - val_loss: 1.3792
Epoch 2/5
95/95 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.3142 - loss: 1.3685 - val_accuracy: 0.3631 - val_loss: 1.3310
Epoch 3/5
95/95 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.5165 - loss: 1.1133 - val_accuracy: 0.8718 - val_loss: 0.4615
Epoch 4/5
95/95 ━━━━━━━━━━━━━━━━━━━━ 209s 2s/step - accuracy: 0.8323 - loss: 0.4796 - val_accuracy: 0.9655 - val_loss: 0.1460
Epoch 5/5
95/95 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.9258 - loss: 0.2118 - val_accuracy: 0.9946 - val_loss: 0.0359
29/29 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step

📊 CNN Classification Report:

                                                                  precision    recall  f1-score   support

      ECG Images of Myocardial Infarction Patients (240x12=2880)       1.00      1.00      1.00       239
     ECG Images of Patient that have History of MI (172x12=2064)       0.98      0.99      0.99  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Lab 12

Question A1. Implement a RNN and bi-directional RNN on your project. Additionally, implement LSTM on your project. 

In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Bidirectional, Dense
from sklearn.metrics import classification_report

In [25]:
# ----------------------------------------
# ✅ Environment Setup (CPU only + clean logs)
# ----------------------------------------
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Disable GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'   # Suppress TensorFlow INFO/WARN/ERROR

# ----------------------------------------
# ✅ Configuration
# ----------------------------------------
IMG_SIZE = (180, 180)
BATCH_SIZE = 32
EPOCHS = 15
train_dir = '/kaggle/input/ecg-analysis/ECG_DATA/train'
test_dir = '/kaggle/input/ecg-analysis/ECG_DATA/test'

In [26]:
# ----------------------------------------
# ✅ Load and Flatten Images into Sequences
# ----------------------------------------
def load_sequence_data():
    datagen = ImageDataGenerator(rescale=1./255)
    train_gen = datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
    test_gen = datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

    def flatten_batches(gen):
        X, y = [], []
        for i in range(len(gen)):
            x_batch, y_batch = gen[i]
            x_flat = x_batch.reshape(x_batch.shape[0], IMG_SIZE[0], IMG_SIZE[1] * 3)  # Flatten image into sequence
            X.append(x_flat)
            y.append(y_batch)
            if len(X) * BATCH_SIZE >= gen.samples:
                break
        return np.concatenate(X), np.concatenate(y)

    X_train, y_train = flatten_batches(train_gen)
    X_test, y_test = flatten_batches(test_gen)
    return X_train, y_train, X_test, y_test, train_gen.class_indices

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Bidirectional, Dense

# ----------------------------------------
# ✅ RNN Variants
# ----------------------------------------

# ✅ Basic RNN
def build_rnn(input_shape, num_classes):
    return Sequential([
        Input(shape=input_shape),             # Expects (180, 540)
        SimpleRNN(64, activation='tanh'),
        Dense(num_classes, activation='softmax')
    ])

# ✅ Bidirectional RNN
def build_birnn(input_shape, num_classes):
    return Sequential([
        Input(shape=input_shape),             
        Bidirectional(SimpleRNN(64, activation='tanh')),
        Dense(num_classes, activation='softmax')
    ])

# ✅ LSTM
def build_lstm(input_shape, num_classes):
    return Sequential([
        Input(shape=input_shape),             
        LSTM(64, activation='tanh'),
        Dense(num_classes, activation='softmax')
    ])

In [28]:
# ----------------------------------------
# ✅ Train and Evaluate
# ----------------------------------------
def train_and_evaluate(name, model_fn, X_train, y_train, X_test, y_test, label_map):
    print(f"\n🚀 Training: {name}...\n")
    model = model_fn((IMG_SIZE[0], IMG_SIZE[1] * 3), y_train.shape[1])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_test, y_test), verbose=1)

    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)

    print(f"\n📊 Classification Report for {name}:\n")
    print(classification_report(y_true, y_pred, target_names=list(label_map.keys()), zero_division=0))

# ----------------------------------------
# ✅ Main
# ----------------------------------------
if __name__ == "__main__":
    X_train, y_train, X_test, y_test, label_map = load_sequence_data()

    train_and_evaluate("Simple RNN", build_rnn, X_train, y_train, X_test, y_test, label_map)
    train_and_evaluate("Bi-directional RNN", build_birnn, X_train, y_train, X_test, y_test, label_map)
    train_and_evaluate("LSTM", build_lstm, X_train, y_train, X_test, y_test, label_map)

Found 3023 images belonging to 4 classes.
Found 928 images belonging to 4 classes.

🚀 Training: Simple RNN...

Epoch 1/15
95/95 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.2967 - loss: 1.4241 - val_accuracy: 0.2575 - val_loss: 1.3826
Epoch 2/15
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3206 - loss: 1.3643 - val_accuracy: 0.3060 - val_loss: 1.3814
Epoch 3/15
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.2951 - loss: 1.3646 - val_accuracy: 0.2575 - val_loss: 1.3899
Epoch 4/15
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.3021 - loss: 1.3655 - val_accuracy: 0.2575 - val_loss: 1.3877
Epoch 5/15
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.3102 - loss: 1.3641 - val_accuracy: 0.2575 - val_loss: 1.3770
Epoch 6/15
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3115 - loss: 1.3677 - val_accuracy: 0.3060 - val_loss: 1.3774
Epoch 7/15
95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3098 - loss: 1.3716 - val_accuracy: 0.2575 - val_loss: 1.3968
